<a href="https://colab.research.google.com/github/Kelly-Lima25/TCC/blob/main/TCC_Utilizando_os_conceitos_de_PLN_para_extrair_palavras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usando a biblioteca ruppell
A biblioteca permite a extração de texto de arquivos pdf e converte para dataframe.


In [ ]:
!pip install ruppell

In [ ]:
import ruppell

ruppell.setup_language(language='por') #Configurando o idioma para a linguagem do arquivo pdf

#Montando o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#text = ruppell.pdf_to_string(file_path='/content/drive/MyDrive/tcc/tcc/brasil/DOC-SF238339076816-20230503.pdf')
#text

# Criando o dataframe a partir dos arquivos pdf.

Por continente

In [ ]:
data_America = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-doc/América')

data_Africa = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-doc/África')

data_Asia = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-doc/Ásia')

data_Europa = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-doc/Europa')


Por países

In [ ]:
data_Brasil = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/brasil')

data_China = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/china')

data_Egito = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/egito')

data_EUA = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/eua')

data_Singapura = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/singapura')

data_UE = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/ue')

data_Vietna = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/vietna')

data_Ruanda = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/ruanda')

data_ReinoUnido = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/tcc/tcc-2/ru')


# Tratamento das palavras

Utilizando a biblioteca NLTK para fazer a tokenização das palavras

In [ ]:
import nltk

nltk.download("all")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
from nltk import tokenize


**1º tratamento:**  Tokenizando todas as palavras do texto, retirando palavras irrelevantes(ex: de, na, que, em...) e deixando o texto em minúsculo.

In [ ]:

def token_min(texto):
  palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
  frase_processada = list()

  token_espaco = tokenize.WhitespaceTokenizer() #Separa cada palavra do texto com espaço

  for doc in texto["text_raw"]:
      nova_frase = list()
      doc = doc.lower()
      palavras_texto = token_espaco.tokenize(doc)
      for palavra in palavras_texto:
          if palavra not in palavras_irrelevantes:
              nova_frase.append(palavra)
      frase_processada.append(' '.join(nova_frase))

  texto["tratamento_1"] = frase_processada

Importando a biblioteca re e definindo a função remover para substituir a regex dos tratamentos 2, 3 e 4 por um espaço em branco

In [ ]:
import re

In [ ]:
def remover(textos, regex):
    if type(textos) == str:
        return regex.sub("", textos)
    else:
        return [regex.sub("", texto) for texto in textos]

**2º tratamento:** Remoção de pontuação utilizando a biblioteca REGEX

In [ ]:
regex_pontuacao = r"[^\w\s]"
regex_pontuacao_epecial = r'[º]+'

regex_pontu = re.compile("|".join([regex_pontuacao, regex_pontuacao_epecial]))


In [ ]:
def remocao(texto, coluna_texto):
  remocao_pontuacao = remover(coluna_texto, regex_pontu)
  texto["tratamento_2"] = remocao_pontuacao


**3º Tratamento:** Removendo os acentos

In [ ]:
!pip install unidecode

import unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
def sem_acento(texto):
  sem_acentos = [unidecode.unidecode(textos) for textos in texto["tratamento_2"]]
  texto["tratamento_3"] = sem_acentos

4º Tratamento: Removendo dígitos e numeração romana

In [ ]:
def remover_numero(texto):
  #regex_padrao1 = r'\b(7 5  1 1 0 2 0 2  2 0  4 0  a c t n s r p 0 2 0 2  1 2 n l p)\b'
  regex_romano = r'\b(xc|xl|l?x?x?x?)(ix|iv|v?i{0,3})+\b'
  regex_numeral = r'(\d+)'

  regex_digito = re.compile("|".join([regex_romano, regex_numeral]))

  texto_sem_numeracao = remover(texto.tratamento_3, regex_digito)

  texto["tratamento_4"] = texto_sem_numeracao



5º Tratamento: Removendo os espaços em branco

In [ ]:

regex_espaco = re.compile(r" +")

def substituir_por_espaco(textos, regex):
    if type(textos) == str:
        return regex.sub(" ", textos)
    else:
        return [regex.sub(" ", texto) for texto in textos]

def remover_espaco(texto):
  remocao_de_espaco = substituir_por_espaco(texto.tratamento_4, regex_espaco)

  texto["tratamento_5"] = remocao_de_espaco

6º Tratamento: Lematização/tratamento individual

In [ ]:
import spacy

# Baixe o modelo 'pt_core_news_sm'
spacy.cli.download("pt_core_news_sm")

# Carregue o modelo para português
nlp = spacy.load('pt_core_news_sm')

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Função para lematizar um texto
def lemmatize_text(texto):
    doc = nlp(str(texto))
    palavra_nao_excluida = 'ia'
    lemmas = [token.lemma_ for token in doc  if token.pos_ == 'NOUN' and token.text != palavra_nao_excluida]

    return ' '.join(lemmas)

In [ ]:
regex_Asia = re.compile(r"\b(dados|organizacoes|modelo|podem|pode|sobre|servicos|decisao|modelos|exemplo|decisoes|medidas|devem|estrutura|recursos|uso|tomada|sistema|fornecer|b|usar|maneira|considerar|eou|popular|aumentar|china|interna|incluindo|interessadas|ou|vietname|dentro|melhorar|tomar|usada|acordo|deve|baseada|d|publica|aumentada|consideracoes|manter|c|promover|produtos|garantir|relevantes|processo|conjunto|a|lei|por|artigo|diferentes|base|partes|outras)\b")

regex_America = re.compile(r"\b(sistemas|sistema|art|lei|competente|uso|desta|bem|utilizacao|secretario|dia|diretor|prazo|agencia|data|ordem|medida|programa|autoridade|pratica|capacidade|forma|presidente|modelo|termo|avaliacao|resultado|atividade|setor|recurso|agente|parte|poder|implementacao|base|estrutura|consulta|acesso|acordo|conselho|objetivo|fim|entidade|grupo|paragrafo|despacho|contexto|artigo|pedido|seg|conta|secao|padroe|energia|orientacoe|assunto|conjunto|maneira|caso|processo)\b")

regex_Africa = re.compile(r"\b(d b d n h s|idf fdnl r t fdd dsf cc s g h t s|c e n c c m p u t d r s s t m s n f r m c a n f r m c a t c n g c n t f c n f r m a t c p r c a p d s s s q u s c b a e s n c c s s|s s t m s c m p u t d r s m u t m d p r g r m s n g n h r|c u t r|estrategia|egipto|pode|egito|uso|podem|nivel|paises|sobre|fase|diferentes|atraves|geral|tempo|numero|total|e|menos|principais|bem|todos|outros|setor|internacional|pais|especialmente|escala|ponto|melhorar|relacionadas|classificacao|nova|novo|outra|todo|onde|toda|muitos|desde|necessario|muita|maior|alguns|piloto|ajudar|cada|disso|ano|sendo|qualquer|etc|deve|sendo|devem|sector|longo|parte|tanto|ter|toda|maquina|alem|abaixo)\b")

regex_Europa = re.compile(r"\b(artigo|elevado|conformidade|requisito|medida|comissao|capitulo|ato|avaliacao|caso|procedimento|atividade|colocacao|titulo|termo|utilizacao|finalidade|efeito|estadomembro|termos|parte|base|controlo|registo|ponto|estadosmembro|acordo|norma|notificacao|tecnica|organismo|resultado|c|execucao|tido|prazo|elemento||circunstancia|coima|ambito|fim|conjunto|aquando|tr|copia|tempo|paragrafo|jornal|aspeto|pai|documentacao)\b")



In [ ]:
regex_China = re.compile(r"\b(organizacoes|lei|artigos|medidas|devem|acordo|china|republica|relevantes|popular|disposicoes|administrativos|leis|deve|outras|outros|instituicoes|direitos|realizar|capitulo|tomar|melhorar|uso|sobre|artigo|fornecer|gerar|etc|termos|competentes|competente|relevante|ministerio)\b")

regex_Brasil = re.compile(r"\b(sistemas|sistema|art|lei|competente|uso|desta|bem|utilizacao|medidas|avaliacao|direito|sobre|funcionamento|adocao|forma|paragrafo|unico|incluindo|riscos|meio|termos|previstos|praticas|caso|n|seguintes|pessoa|alto)\b")

regex_EUA = re.compile(r"\b(desenvolver|nacional|secretario|dia|diretor|prazo|agencia|data|ordem|medida|programa|autoridade|pratica|capacidade|forma|presidente|modelo|termo|avaliacao|resultado|atividade|setor|recurso|agente|parte|poder|implementacao|base|estrutura|consulta|acesso|acordo|conselho|objetivo|fim|entidade|grupo|paragrafo|despacho|contexto|artigo|pedido|seg|conta|secao|padroe|energia|orientacoe|assunto|conjunto|maneira|caso|processo|incluindo|dias|sobre|apropriado|partir|conforme|estados|outras|lei|unidos|devera|desta|a|b|c|d|e|f|g|h|j|k|m|deste|modelos|considerar|relacionadas|interna|relevantes|criticas|coordenacao|orientacoes|outros|bem|devem|meio|qualquer|consistente|uso|federal)\b")

regex_Egito = re.compile(r"\b(d b d n h s|idf fdnl r t fdd dsf cc s g h t s|c e n c c m p u t d r s s t m s n f r m c a n f r m c a t c n g c n t f c n f r m a t c p r c a p d s s s q u s c b a e s n c c s s|s s t m s c m p u t d r s m u t m d p r g r m s n g n h r|c u t r|estrategia|egipto|pode|egito|uso|podem|nivel|paises|sobre|fase|diferentes|atraves|geral|tempo|numero|total|e|menos|principais|bem|todos|outros|setor|internacional|pais|especialmente|escala|ponto|melhorar|relacionadas|classificacao|nova|novo|outra|todo|onde|toda|muitos|desde|necessario|muita|maior|alguns|piloto|ajudar|cada|disso|ano|sendo|qualquer|etc|deve|sendo|devem|sector|longo|parte|tanto|ter|toda|maquina|alem|abaixo|investigacao|exemplo|incluindo|forma|portanto)\b")

regex_Singapura = re.compile(r"\b(modelo|podem|pode|decisao|modelos|exemplo|sobre|tomada|garantir|medidas|conjunto|devem|praticas|por|a|b|c|d|e|f|processo|considerar|algoritmos|partes|principios|relevantes|sistemas|diferentes|interessadas|conjunto|organizacao|estrutura|estruturas|decisoes|eou|risco|usar|incluindo|uso|organizacoes)\b")

regex_UE = re.compile(r"\b(inteligencia|artificial|artigo|elevado|conformidade|requisito|medida|comissao|capitulo|ato|avaliacao|caso|procedimento|atividade|colocacao|titulo|termo|utilizacao|finalidade|efeito|termos|parte|base|controlo|registo|ponto|estadosmembros|acordo|norma|notificacao|tecnica|organismo|resultado|c|execucao|tido|prazo|elemento|sistemas|circunstancia|coima|ambito|fim|conjunto|aquando|tr|copia|tempo|paragrafo|jornal|aspeto|pai|documentacao|devem|deve|presente|autoridade|regulamento|n|requisitos|organismos|uniao|estabelecidos|ue|refere|medidas|anexo|notificados|competentes|atividades|nacional|qualquer|atos|incluindo|causa|efeitos|prevista|a|b|d|e|f|g|h|j|k|l|m|n|fornecedores|gestao|sobre|pode|todas|relacao|pedido|disso|conta|utilizador|diretiva|necessarias|podem|quaisquer|seguintes|adequadas|referidos|outras|alinea|cada|nomeadamente|bem|desses|desse|ter|outros|sob|nacionais)\b")

regex_Vietna = re.compile(r"\b(medidas|aplicacoes|sobre|alto|alta|partilha|centro|data|ministerio|nacionais|vietname|implementacao|melhorar|nacional)\b")

regex_Ruanda = re.compile(r"\b(ano|fiscal|nacional|sobre|estabelecer|realizar|numero|resultado)\b")

regex_ReinoUnido = re.compile(r"\b(sobre|cid|reino|unido|forma|podem|regulamentacao|pode|exemplo|incluindo|regulamentares|modelos|existentes|todo|funcoes|funcao)\b")

##Por Continente

1º Tratamento:

In [ ]:
token_min(data_America)
token_min(data_Africa)
token_min(data_Asia)
token_min(data_Europa)


2º Tratamento:

In [ ]:
remocao(data_America, data_America.tratamento_1)
remocao(data_Africa, data_Africa.tratamento_1)
remocao(data_Asia, data_Asia.tratamento_1)
remocao(data_Europa, data_Europa.tratamento_1)


3º Tratamento:

In [ ]:
sem_acento(data_America)
sem_acento(data_Africa)
sem_acento(data_Asia)
sem_acento(data_Europa)


4º Tratamento:

In [ ]:
remover_numero(data_America)
remover_numero(data_Africa)
remover_numero(data_Asia)
remover_numero(data_Europa)

5º Tratamento:

In [ ]:
remover_espaco(data_America)
remover_espaco(data_Africa)
remover_espaco(data_Asia)
remover_espaco(data_Europa)

6º Tratamento:

In [ ]:
tratamento_6 = remover(data_Asia.tratamento_5, regex_Asia)
data_Asia["tratamento_6"] = tratamento_6

In [ ]:
tratamento_6_Africa = remover(data_Africa.tratamento_5, regex_Africa)
data_Africa["tratamento_6"] = tratamento_6_Africa

In [ ]:
data_America["tratamento_6"] = data_America['tratamento_5'].apply(lemmatize_text)
#tratamento_7_America = remover(data_America.tratamento_6, regex_America)
#data_America["tratamento_7"] = tratamento_7_America

In [ ]:
data_America['tratamento_5'][2]

'secao objetivo inteligencia artificial ia possui potencial extraordinario tanto promessas perigos uso responsavel ia potencial ajudar resolver desafios urgentes e tempo tornar mundo prospero produtivo inovador seguro tempo uso irresponsavel pode exacerbar danos sociais fraude discriminacao preconceito desinformacao deslocar enfraquecer trabalhadores sufocar concorrencia representam riscos seguranca nacional aproveitar ia bem concretizar inumeros beneficios exige mitigacao riscos substanciais esforco exige esforco toda sociedade inclua governo sector privado meio academico sociedade civil administracao da maior urgencia governar desenvolvimento utilizacao ia forma segura responsavel e portanto promover abordagem coordenada todo governo federal fazer rapida velocidade capacidades ia avancar obriga estados unidos liderar neste momento prol seguranca economia sociedade final contas ia reflete principios pessoas constroem pessoas utilizam dados sobre quais construida acredito firmemente po

In [ ]:
data_Europa["tratamento_6"] = data_Europa['tratamento_5'].apply(lemmatize_text)
#tratamento_7_Europa = remover(data_Europa.tratamento_6, regex_Europa)
#data_Europa["tratamento_7"] = tratamento_7_Europa

In [ ]:
data_Europa

,file_name,text_raw,tratamento_1,tratamento_2,tratamento_3,tratamento_4,tratamento_5,tratamento_6
0,União Europeia.pdf,"28/04/2024, 18:17\n\na)\n\nb)\n\nc)\n\nd)\n\nR...","28/04/2024, 18:17 a) b) c) d) regras harmoniza...",28042024 1817 a b c d regras harmonizadas colo...,28042024 1817 a b c d regras harmonizadas colo...,a b c d regras harmonizadas colocacao mercad...,a b c d regras harmonizadas colocacao mercado...,c regra colocacao sistema inteligencia sistema...


##Por País

1º Tratamento:

In [ ]:
token_min(data_Brasil)
token_min(data_China)
token_min(data_EUA)
token_min(data_Egito)
token_min(data_Singapura)
token_min(data_UE)
token_min(data_Vietna)
token_min(data_Ruanda)
token_min(data_ReinoUnido)

2º Tratamento:

In [ ]:
remocao(data_Brasil,data_Brasil.tratamento_1)
remocao(data_China, data_China.tratamento_1)
remocao(data_EUA, data_EUA.tratamento_1)
remocao(data_Egito, data_Egito.tratamento_1)
remocao(data_Singapura, data_Singapura.tratamento_1)
remocao(data_UE, data_UE.tratamento_1)
remocao(data_Vietna, data_Vietna.tratamento_1)
remocao(data_Ruanda, data_Ruanda.tratamento_1)
remocao(data_ReinoUnido, data_ReinoUnido.tratamento_1)

3º Tratamento:

In [ ]:
sem_acento(data_Brasil)
sem_acento(data_China)
sem_acento(data_EUA)
sem_acento(data_Egito)
sem_acento(data_Singapura)
sem_acento(data_UE)
sem_acento(data_Vietna)
sem_acento(data_Ruanda)
sem_acento(data_ReinoUnido)

4º Tratamento:

In [ ]:
remover_numero(data_Brasil)
remover_numero(data_China)
remover_numero(data_EUA)
remover_numero(data_Egito)
remover_numero(data_Singapura)
remover_numero(data_UE)
remover_numero(data_Vietna)
remover_numero(data_Ruanda)
remover_numero(data_ReinoUnido)

5º Tratamento:

In [ ]:
remover_espaco(data_Brasil)
remover_espaco(data_China)
remover_espaco(data_EUA)
remover_espaco(data_Egito)
remover_espaco(data_Singapura)
remover_espaco(data_UE)
remover_espaco(data_Vietna)
remover_espaco(data_Ruanda)
remover_espaco(data_ReinoUnido)

In [ ]:
data_ReinoUnido


,file_name,text_raw,tratamento_1,tratamento_2,tratamento_3,tratamento_4,tratamento_5
0,UK regulation-PT-BR.pdf,Machine Translated by Google\n\nUma abordagem...,machine translated by google abordagem pró-ino...,machine translated by google abordagem próinov...,machine translated by google abordagem proinov...,machine translated by google abordagem proinov...,machine translated by google abordagem proinov...


6º Tratamento:

In [ ]:
tratamento_6_Brasil = remover(data_Brasil.tratamento_5, regex_Brasil)
data_Brasil["tratamento_6"] = tratamento_6_Brasil

In [ ]:
tratamento_6_China = remover(data_China.tratamento_5, regex_China)
data_China["tratamento_6"] = tratamento_6_China

In [ ]:
tratamento_6_EUA = remover(data_EUA.tratamento_5, regex_EUA)
data_EUA["tratamento_6"] = tratamento_6_EUA

In [ ]:
tratamento_6_Egito = remover(data_Egito.tratamento_5, regex_Egito)
data_Egito["tratamento_6"] = tratamento_6_Egito

In [ ]:
tratamento_6_Singapura = remover(data_Singapura.tratamento_5, regex_Singapura)
data_Singapura["tratamento_6"] = tratamento_6_Singapura

In [ ]:
tratamento_6_UE = remover(data_UE.tratamento_5, regex_UE)
data_UE["tratamento_6"] = tratamento_6_UE

In [ ]:
tratamento_6_Vietna = remover(data_Vietna.tratamento_5, regex_Vietna)
data_Vietna["tratamento_6"] = tratamento_6_Vietna

In [ ]:
tratamento_6_Ruanda = remover(data_Ruanda.tratamento_5, regex_Ruanda)
data_Ruanda["tratamento_6"] = tratamento_6_Ruanda

In [ ]:
tratamento_6_ReinoUnido = remover(data_ReinoUnido.tratamento_5, regex_ReinoUnido)
data_ReinoUnido["tratamento_6"] = tratamento_6_ReinoUnido

In [ ]:
data_Brasil["tratamento_6"][1]

'  estabelece principios direitos deveres instrumentos governanca  inteligencia artificial brasil determina diretrizes atuacao uniao estados distrito federal municipios pessoas fisicas juridicas  publico privado entes personalidade juridica relacao materia  fins   considerase  inteligencia artificial  baseado processo computacional pode determinado conjunto objetivos definidos homem fazer previsoes recomendacoes tomar decisoes influenciam ambientes reais virtuais ciclo vida  inteligencia artificial composto fases sequenciais nao planejamento design coleta processamento dados construcao modelo verificacao validacao implantacao operacao monitoramento conhecimento inteligencia artificial habilidades recursos dados codigos algoritmos pesquisas programas treinamento governanca melhores  necessarios conceber gerir entender participar ciclo vida  agentes inteligencia artificial pessoas fisicas juridicas  publico privado entes personalidade juridica assim considerados a agentes desenvolvimento

# Visualização das palavras

O **WordCloud** é uma biblioteca de visualizção de palavras que aparecem com mais freqência no texto, sua saída é um objeto e para transformar este objeto em uma imagem, utiliza-se a biblioteca **matplotlib** para plotar a imagem.

In [ ]:
#Biblioteca para plotar imagem
%matplotlib inline

In [ ]:
#importando a biblioteca de visualizção das palavaras mais citadas nos documentos
from wordcloud import WordCloud

Plotagem da imagem

In [ ]:

import matplotlib.pyplot as plt

def visualizacao(texto,coluna_texto):
  todas_palavras = ' '.join([textos for textos in texto[coluna_texto]]) #O wordCloud funciona com lista, para isso é preciso
                                                                          #juntar todos os textos desejados em uma única lista

  nuvem_palavras = WordCloud(width=800, height=500, max_font_size= 110, collocations= False).generate(todas_palavras)

  plt.figure(figsize=(10,7))
  plt.imshow(nuvem_palavras, interpolation='bilinear')
  plt.axis("off")
  #plt.savefig('imagem4.png', format='png')
  plt.show()



In [ ]:
#img1 = visualizacao(data_America, "tratamento_7")

#img2 = visualizacao(data_Africa, "tratamento_6")
#img3 = visualizacao(data_Asia, "tratamento_6")
img4 = visualizacao(data_Europa, "tratamento_7")

In [ ]:
import cv2

In [ ]:
fig = plt.figure(figsize=(16, 12))
rows = 2
columns = 2

imagem1 = cv2.imread("imagem.png")
imagem2 = cv2.imread("imagem2.png")
imagem3 = cv2.imread("imagem3.png")
imagem4 = cv2.imread("imagem4.png")
# showing image
fig.add_subplot(rows, columns, 1)
plt.imshow(imagem1)
plt.axis('off')
plt.title("América")

fig.add_subplot(rows, columns, 2)
plt.imshow(imagem2)
plt.axis('off')
plt.title("África")


# Adds a subplot at the 3rd position
fig.add_subplot(rows, columns, 3)

# showing image
plt.imshow(imagem3)
plt.axis('off')
plt.title("Ásia")


# Adds a subplot at the 3rd position
fig.add_subplot(rows, columns, 4)

# showing image
plt.imshow(imagem4)
plt.axis('off')
plt.title("Europa")

In [ ]:
import pandas as pd

# Tabela por continente

In [ ]:
token_espaco = tokenize.WhitespaceTokenizer()
todas_palavras_America = ' '.join([texto for texto in data_America["tratamento_6"]])
frequencia_America = nltk.FreqDist(token_espaco.tokenize(todas_palavras_America))

todas_palavras_Asia = ' '.join([texto for texto in data_Asia["tratamento_6"]])
frequencia_Asia = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Asia))

todas_palavras_Africa = ' '.join([texto for texto in data_Africa["tratamento_6"]])
frequencia_Africa = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Africa))

todas_palavras_Europa = ' '.join([texto for texto in data_Europa["tratamento_6"]])
frequencia_Europa = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Europa))

In [ ]:
df_frequencia = pd.DataFrame({"América":list(frequencia_America.keys()), "Frequência": list(frequencia_America.values())})
lista_America = df_frequencia.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Asia = pd.DataFrame({"Ásia":list(frequencia_Asia.keys()), "Frequência": list(frequencia_Asia.values())})
lista_Asia = df_frequencia_Asia.nlargest(columns = "Frequência", n = 12)

In [ ]:
df_frequencia_Africa = pd.DataFrame({"África":list(frequencia_Africa.keys()), "Frequência": list(frequencia_Africa.values())})
lista_Africa = df_frequencia_Africa.nlargest(columns = "Frequência", n = 12)

In [ ]:
df_frequencia_Europa = pd.DataFrame({"Europa":list(frequencia_Europa.keys()), "Frequência": list(frequencia_Europa.values())})
lista_Europa = df_frequencia_Europa.nlargest(columns = "Frequência", n = 12)

In [ ]:
from IPython.display import HTML
def  side_by_side_cont ( *dfs ):
  html = '<div style="display:flex">'
  for df in dfs:
      html += '<div style="margin-right: 2em">'
      html += df.to_html()
      html += '</div>'
  html += '</div>'
  display(HTML(html))

In [ ]:
side_by_side_cont(lista_America, lista_Asia,lista_Africa, lista_Europa)

,América,Frequência
4,sistema,197
5,inteligencia,189
42,risco,129
708,secretario,107
32,seguranca,103
3,uso,98
150,dia,94
709,diretor,94
2,desenvolvimento,92
149,prazo,92


In [ ]:
side_by_side_cont(lista_America, lista_Asia,lista_Africa, lista_Europa)

,América,Frequência
2,inteligencia,189
35,risco,129
26,seguranca,103
1,desenvolvimento,92
3,direito,86
257,tecnologia,80
6,pessoa,76
459,trabalho,51
522,governo,51
172,saude,34


#Tabela por país

In [ ]:
token_espaco = tokenize.WhitespaceTokenizer()
todas_palavras_Brasil = ' '.join([texto for texto in data_Brasil["tratamento_6"]])
frequencia_Brasil = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Brasil))

todas_palavras_China = ' '.join([texto for texto in data_China["tratamento_6"]])
frequencia_China = nltk.FreqDist(token_espaco.tokenize(todas_palavras_China))

todas_palavras_EUA = ' '.join([texto for texto in data_EUA["tratamento_6"]])
frequencia_EUA = nltk.FreqDist(token_espaco.tokenize(todas_palavras_EUA))

todas_palavras_Egito = ' '.join([texto for texto in data_Egito["tratamento_6"]])
frequencia_Egito = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Egito))

todas_palavras_Singapura = ' '.join([texto for texto in data_Singapura["tratamento_6"]])
frequencia_Singapura = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Singapura))

todas_palavras_UE = ' '.join([texto for texto in data_UE["tratamento_6"]])
frequencia_UE = nltk.FreqDist(token_espaco.tokenize(todas_palavras_UE))

todas_palavras_Vietna = ' '.join([texto for texto in data_Vietna["tratamento_6"]])
frequencia_Vietna = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Vietna))

todas_palavras_Ruanda = ' '.join([texto for texto in data_Ruanda["tratamento_6"]])
frequencia_Ruanda = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Ruanda))

todas_palavras_ReinoUnido = ' '.join([texto for texto in data_ReinoUnido["tratamento_6"]])
frequencia_ReinoUnido = nltk.FreqDist(token_espaco.tokenize(todas_palavras_ReinoUnido))

In [ ]:
df_frequencia_Brasil = pd.DataFrame({"Brasil":list(frequencia_Brasil.keys()), "Frequência": list(frequencia_Brasil.values())})
lista_Brasil = df_frequencia_Brasil.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_China = pd.DataFrame({"China":list(frequencia_China.keys()), "Frequência": list(frequencia_China.values())})
lista_China = df_frequencia_China.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_EUA = pd.DataFrame({"EUA":list(frequencia_EUA.keys()), "Frequência": list(frequencia_EUA.values())})
lista_EUA = df_frequencia_EUA.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Egito = pd.DataFrame({"Egito":list(frequencia_Egito.keys()), "Frequência": list(frequencia_Egito.values())})
lista_Egito = df_frequencia_Egito.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Singapura = pd.DataFrame({"Singapura":list(frequencia_Singapura.keys()), "Frequência": list(frequencia_Singapura.values())})
lista_Singapura = df_frequencia_Singapura.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_UE = pd.DataFrame({"UE":list(frequencia_UE.keys()), "Frequência": list(frequencia_UE.values())})
lista_UE = df_frequencia_UE.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Vietna = pd.DataFrame({"Vietnã":list(frequencia_Vietna.keys()), "Frequência": list(frequencia_Vietna.values())})
lista_Vietna = df_frequencia_Vietna.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Ruanda = pd.DataFrame({"Ruanda":list(frequencia_Ruanda.keys()), "Frequência": list(frequencia_Ruanda.values())})
lista_Ruanda = df_frequencia_Ruanda.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_ReinoUnido = pd.DataFrame({"Reino Unido":list(frequencia_ReinoUnido.keys()), "Frequência": list(frequencia_ReinoUnido.values())})
lista_ReinoUnido = df_frequencia_ReinoUnido.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
lista_Vietna

,Vietnã,Frequência
2,ia,119
26,desenvolvimento,38
151,dados,38
25,investigacao,33
67,promover,32
27,aplicacao,28
62,desenvolver,25
91,computacao,22
3,tecnologia,19
35,recursos,17


In [ ]:
from IPython.display import HTML
def  side_by_side_country ( *dfs ):
  html = '<div style="display:grid; grid-template-columns: repeat(4,300px);">'
  for df in dfs:
      html += '<div style="margin-right: 2em">'
      html += df.to_html()
      html += '</div>'
  html += '</div>'
  display(HTML(html))

In [ ]:
side_by_side_country(lista_Brasil, lista_China, lista_Egito, lista_EUA, lista_Singapura, lista_UE, lista_Vietna, lista_Ruanda, lista_ReinoUnido)

,Brasil,Frequência
4,inteligencia,196
5,artificial,195
56,dados,55
23,direitos,48
16,desenvolvimento,45
562,risco,44
250,pessoas,36
0,f,33
1,s,31
193,autoridade,30


#Referências

[Linguagem Natural parte 1: NLP com análise de sentimentok](https://www.alura.com.br/curso-online-introducao-a-nlp-com-analise-de-sentimento)

[Linguagem Natural parte 2: continuando com a análise de sentimento](https://www.alura.com.br/curso-online-nlp-com-analise-de-sentimento)

[NLP: regex e modelos de linguagem](https://www.alura.com.br/curso-online-nlp-modelos-linguagem)


[Expressões regularem em Python](https://www.acervolima.com.br/2021/05/expressoes-regulares-em-python.html)

[Lemmatization vs. stemming: quando usar cada uma?](https://www.alura.com.br/artigos/lemmatization-vs-stemming-quando-usar-cada-uma)

[Displaying Multiple DataFrames Side By Side in Jupyter Lab/Notebook](https://python.plainenglish.io/displaying-multiple-dataframes-side-by-side-in-jupyter-lab-notebook-9a4649a4940)